1. Installing dependencies

In [14]:
!pip install streamlit pyngrok scikit-learn xgboost pandas joblib --quiet


2. Importing libraries

In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib

3. Creating data, preprocessing, training model, and saving feature order

In [16]:
# Generate synthetic data
np.random.seed(42)
data = {
    'floor_area': np.random.uniform(50, 5000, 1000),
    'num_floors': np.random.randint(1, 20, 1000),
    'concrete_volume': np.random.uniform(10, 1000, 1000),
    'steel_mass': np.random.uniform(100, 5000, 1000),
    'climate_zone': np.random.choice(['Tropical', 'Temperate', 'Continental'], 1000),
    'transport_radius': np.random.uniform(10, 500, 1000),
    'co2e': np.random.uniform(5000, 500000, 1000)
}
df = pd.DataFrame(data)
df = pd.get_dummies(df, columns=['climate_zone'])

# Prepare features and target
X = df.drop('co2e', axis=1)
y = df['co2e']

# Save feature order for prediction
feature_order = list(X.columns)
joblib.dump(feature_order, 'feature_order.pkl')

# Train model
model = xgb.XGBRegressor(n_estimators=100)
model.fit(X, y)
joblib.dump(model, 'carbon_model.pkl')


['carbon_model.pkl']

4. Writing the Streamlit app, enforcing feature order

In [17]:
app_code = '''
import streamlit as st
import joblib
import pandas as pd

model = joblib.load('carbon_model.pkl')
feature_order = joblib.load('feature_order.pkl')

st.title("🏗️ Carbon Footprint Predictor")
with st.form("my_form"):
    floor_area = st.number_input("Floor Area (m²)", 50, 5000, 1000)
    num_floors = st.number_input("Number of Floors", 1, 50, 5)
    concrete = st.number_input("Concrete Volume (m³)", 0.0, 1000.0, 500.0)
    steel = st.number_input("Steel Mass (kg)", 0.0, 5000.0, 2000.0)
    climate = st.selectbox("Climate Zone", ["Tropical", "Temperate", "Continental"])
    transport = st.slider("Transport Radius (km)", 10, 500, 100)

    if st.form_submit_button("Predict"):
        input_dict = {
            'floor_area': floor_area,
            'num_floors': num_floors,
            'concrete_volume': concrete,
            'steel_mass': steel,
            'transport_radius': transport,
            'climate_zone_Continental': 1 if climate == 'Continental' else 0,
            'climate_zone_Temperate': 1 if climate == 'Temperate' else 0,
            'climate_zone_Tropical': 1 if climate == 'Tropical' else 0
        }
        # Build DataFrame and set correct column order
        input_data = pd.DataFrame([input_dict])[feature_order]
        prediction = model.predict(input_data)[0]
        st.metric("Predicted CO₂e", f"{prediction:,.0f} kg")
'''

with open('app.py', 'w') as f:
    f.write(app_code)


5. Setting the ngrok authtoken and launching the app

In [25]:
from pyngrok import ngrok, conf

# Cleanup previous tunnels
ngrok.kill()

# Configure to use only 1 tunnel
conf.get_default().region = "us"  # Optional: set region for consistency

# Set your authtoken
ngrok.set_auth_token("2xxXZQI2TsbfH0gqBIj0DPwGGRM_23rTk4zvziycwrkDjF8Ab")  # Replace with your token

# Start Streamlit and ngrok
!streamlit run app.py --server.port 8501 &>/dev/null &
public_url = ngrok.connect("http://localhost:8501", bind_tls=True)  # Single HTTPS tunnel
print("App URL:", public_url.public_url)


App URL: https://6966-34-53-119-26.ngrok-free.app
